In [1]:
from pandas import read_csv, DataFrame

NUM_PARTICIPANTS = 24
CONDITIONS = ['A_Baseline', 'B_HumanHorizontal', 'C_HumanVertical', 'D_RobotHorizontal']
# CONDITIONS = ['A_Baseline']

In [2]:
##########################################################################################
def get_raw_data():
    
    my_file_dir = 'c:\\Users\\micha\\OneDrive\\My_GitHub_Repos\\robot-x-ar\\data\\task_performance\\collated_data.csv'
    big_df = read_csv(my_file_dir)
    df = big_df[['ParticipantID', 'Condition', 'Trial', 'TargetID', 'Timestamp']]
    
    print("\n Finished reading raw csv file! \n")
    
    return df

##########################################################################################
def calc_timestamp_diff(t1, t2):
    # hours
    h1 = int(str(t1)[8:10])
    h2 = int(str(t2)[8:10])
    hour_diff = h2 - h1
    # minutes
    m1 = int(str(t1)[10:12])
    m2 = int(str(t2)[10:12])
    min_diff = hour_diff*60 + (m2-m1)
    # seconds
    s1 = int(str(t1)[12:])/1e4
    s2 = int(str(t2)[12:])/1e4
    sec_diff = min_diff*60 + (s2-s1)
    
    return round(sec_diff, 4)
    
##########################################################################################
def process_timestamp_lst(lst: list, start_idx: int):
    diff_lst = []
    for i in range(start_idx, len(lst)-1):
        diff = calc_timestamp_diff(lst[i], lst[i+1])
        diff_lst.append(diff)
    return diff_lst

In [3]:
task_df = get_raw_data()
task_df


 Finished reading raw csv file! 



,ParticipantID,Condition,Trial,TargetID,Timestamp
0,10,B_HumanHorizontal,1,3,202403271847422787
1,10,B_HumanHorizontal,1,8,202403271847573776
2,10,B_HumanHorizontal,1,2,202403271848210167
3,10,B_HumanHorizontal,1,7,202403271848539901
4,10,B_HumanHorizontal,1,1,202403271849183157
...,...,...,...,...,...
2107,9,B_HumanHorizontal,2,0,202403271733481234
2108,9,B_HumanHorizontal,2,5,202403271734028316
2109,9,B_HumanHorizontal,2,10,202403271734191390
2110,9,B_HumanHorizontal,2,4,202403271734344388


In [5]:
part_id_lst = []
condition_lst = []
trial_id_lst = []
average_mt_lst = []
average_skip_first_mt_lst = []

for part_id in range(1, NUM_PARTICIPANTS+1):
    part_data = task_df[task_df['ParticipantID']==part_id]
    for condition in CONDITIONS:
        cond_data = part_data[part_data['Condition']==condition]
        for trial_id in range(1, 3):
            trial_data = cond_data[cond_data['Trial']==trial_id]
            timestamp_lst = trial_data['Timestamp'].tolist()
            mt_lst = process_timestamp_lst(timestamp_lst, start_idx=0)
            average_mt = sum(mt_lst) / len(mt_lst)
            mt_skip_first_lst = process_timestamp_lst(timestamp_lst, start_idx=1)
            average_skip_first_mt = sum(mt_skip_first_lst) / len(mt_skip_first_lst)
            # add to list storages
            part_id_lst.append(part_id)
            condition_lst.append(condition[0])
            trial_id_lst.append(trial_id)
            average_mt_lst.append(average_mt)
            average_skip_first_mt_lst.append(average_skip_first_mt)
            

###### write to new (cleaned) dataframe ######
    
# generate new dataframe
df_dict = {
    'pid': part_id_lst,
    'condition': condition_lst,
    'trial': trial_id_lst,
    'average_mt': average_mt_lst,
    'average_skip_first_mt': average_skip_first_mt_lst
}
cleaned_df = DataFrame(df_dict)

# write new dataframe to csv file
dest_path = 'c:\\Users\\micha\\OneDrive\\My_GitHub_Repos\\robot-x-ar\\data\\task_performance\\movement_time.csv'
cleaned_df.to_csv(dest_path, index=False)

print(" Successfully written pre-processed data to csv file! \n")

 Successfully written pre-processed data to csv file! 

